In [1]:
import pandas as pd
df_participant = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv')

In [2]:
df_participant

,participant_id,first_name,last_name,birth_date,address,phone_number,country,institute,occupation,register_time
0,bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a,Citra,Nurdiyanti,05 Feb 1991,"Gg. Monginsidi No. 08\nMedan, Aceh 80734",(0151) 081 2706,Georgia,UD Prakasa Mandasari,Business Intelligence Engineer,1.617634e+09
1,7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3,Aris,Setiawan,11 Jan 1993,"Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434",+62 (036) 461 7027,Korea Utara,Universitas Diponegoro,Frontend Engineer,1.617634e+09
2,19582d7f-b824-4fe5-a517-d5bf573fc768,Cornelia,Handayani,31 Jul 1993,"Jalan Kebonjati No. 0\nAmbon, SS 57739",089 833 6695,Komoro,UD Hardiansyah Puspasari,Business Analyst,1.617634e+09
3,aeb6d817-25f3-4867-8a74-8d92e0a0f633,Soleh,Rajasa,04 Nov 1991,"Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156",+62 (418) 329-4756,Eritrea,Perum Tampubolon Yuliarti,DevOps Engineer,1.617634e+09
4,1fdabdd9-5444-4c97-87b2-fe8833ad0d27,Vivi,Astuti,22 Jan 2003,"Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...",0812511835,Aljazair,PT Hardiansyah Rahimah,Data Analyst,1.617634e+09
...,...,...,...,...,...,...,...,...,...,...
4995,5f084245-58cb-4360-8cff-112f06f7214e,Mitra,Latupono,27 Apr 2000,"Jl. Monginsidi No. 5\nCirebon, BB 50268",+62 (050) 795-8398,Hongaria,Institut Teknologi Sepuluh Nopember,Business Intelligence Engineer,1.617634e+09
4996,fc6e142a-3f66-4330-9975-8c1dc7f9bb03,Indah,Rahmawati,06 Mar 1996,"Gang Joyoboyo No. 73\nBogor, BT 57434",+62 (0443) 963 0055,Norwegia,CV Safitri Hastuti Tbk,Data Engineer,1.617634e+09
4997,4521ff9f-4aec-401b-84aa-a4e6fc6549a4,Irsad,Najmudin,14 Feb 2003,"Jalan PHH. Mustofa No. 0\nTanjungbalai, MU 95312",+62 (0408) 216-7689,Ghana,Universitas Katolik Widya Manadala,Fullstack Engineer,1.617634e+09
4998,1c91596e-427c-44a7-b7d0-974f44e18381,Carla,Nasyidah,22 Aug 1994,"Jl. H.J Maemunah No. 955\nManado, Kepulauan Ba...",+62 (0751) 380 3352,Gambia,CV Permadi (Persero) Tbk,Fullstack Engineer,1.617634e+09


In [3]:
#extract kode pos dari field alamat. kode pos berada di paling akhir dari alamat 
df_participant['postal_code'] = df_participant['address'].str.extract(r'(\d+)$') 

In [4]:
df_participant['postal_code'].head()

0    80734
1    09434
2    57739
3    76156
4    70074
Name: postal_code, dtype: object

In [6]:
#extract kota dari field alamat. kota merupakan sekumpulan karakter yang terdapat setelah nomor jalan diikuti dengan \n (newline character) 
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)')
df_participant['city'].head()

0           Medan
1      Prabumulih
2           Ambon
3    Lubuklinggau
4          Kediri
Name: city, dtype: object

In [7]:
#buat field link github dengan susunan nama depan dan nama belakang yang sudah di lowercase
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()

In [8]:
df_participant['github_profile'].head()

0      https://github.com/citranurdiyanti
1         https://github.com/arissetiawan
2    https://github.com/corneliahandayani
3          https://github.com/solehrajasa
4           https://github.com/viviastuti
Name: github_profile, dtype: object

In [9]:
#ganti +62/62 menjadi 0
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0')
#bersihkan tanda () dan -
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '')
#hilangkan whitespace
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'\s+', '')
df_participant['cleaned_phone_number'].head()

0    01510812706
1    00364617027
2     0898336695
3    04183294756
4     0812511835
Name: cleaned_phone_number, dtype: object

In [10]:
#buat field nama tim yang merupakan gabungan nilai dari kolom first_name, last_name, country dan institute.
def func(col):
  #extrat huruf pertama dari nama depan dan belakang
    abbrev_name = "%s%s"%(col['first_name'][0],col['last_name'][0]) 
    country = col['country']
    abbrev_institute = '%s'%(''.join(list(map(lambda word: word[0], col['institute'].split())))) #Singkatan dari value di kolom institute
    return "%s-%s-%s"%(abbrev_name,country,abbrev_institute)

df_participant['team_name'] = df_participant.apply(func, axis=1)
df_participant['team_name'].head()

0       CN-Georgia-UPM
1    AS-Korea Utara-UD
2        CH-Komoro-UHP
3       SR-Eritrea-PTY
4      VA-Aljazair-PHR
Name: team_name, dtype: object

In [11]:
#buat email dari nama depan, nama belakang, universitas/institut 
def func(col):
	first_name_lower = col['first_name'].lower()
	last_name_lower = col['last_name'].lower()
	institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split()))) #Singkatan dari nama perusahaan dalam lowercase

	if 'Universitas' in col['institute']:
		if len(col['country'].split()) > 1: #Kondisi untuk mengecek apakah jumlah kata dari country lebih dari 1
			country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
		else:
			country = col['country'][:3].lower()
		return "%s%s@%s.ac.%s"%(first_name_lower,last_name_lower,institute,country)

	return "%s%s@%s.com"%(first_name_lower,last_name_lower,institute)

df_participant['email'] = df_participant.apply(func, axis=1)
df_participant['email'].head()

0      citranurdiyanti@upm.com
1        arissetiawan@ud.ac.ku
2    corneliahandayani@uhp.com
3          solehrajasa@pty.com
4           viviastuti@phr.com
Name: email, dtype: object

In [12]:
#reformat tgl lahir
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format='%d %b %Y')

In [13]:
df_participant['birth_date'].head()

0   1991-02-05
1   1993-01-11
2   1993-07-31
3   1991-11-04
4   2003-01-22
Name: birth_date, dtype: datetime64[ns]

In [14]:
#reformat tgl pendaftaran
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')
df_participant['register_at'].head()

0   2021-04-05 14:47:26
1   2021-04-05 14:46:58
2   2021-04-05 14:47:15
3   2021-04-05 14:47:14
4   2021-04-05 14:46:50
Name: register_at, dtype: datetime64[ns]

In [15]:
df_participant.head()

,participant_id,first_name,last_name,birth_date,address,phone_number,country,institute,occupation,register_time,postal_code,city,github_profile,cleaned_phone_number,team_name,email,register_at
0,bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a,Citra,Nurdiyanti,1991-02-05,"Gg. Monginsidi No. 08\nMedan, Aceh 80734",(0151) 081 2706,Georgia,UD Prakasa Mandasari,Business Intelligence Engineer,1.617634e+09,80734,Medan,https://github.com/citranurdiyanti,01510812706,CN-Georgia-UPM,citranurdiyanti@upm.com,2021-04-05 14:47:26
1,7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3,Aris,Setiawan,1993-01-11,"Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434",+62 (036) 461 7027,Korea Utara,Universitas Diponegoro,Frontend Engineer,1.617634e+09,09434,Prabumulih,https://github.com/arissetiawan,00364617027,AS-Korea Utara-UD,arissetiawan@ud.ac.ku,2021-04-05 14:46:58
2,19582d7f-b824-4fe5-a517-d5bf573fc768,Cornelia,Handayani,1993-07-31,"Jalan Kebonjati No. 0\nAmbon, SS 57739",089 833 6695,Komoro,UD Hardiansyah Puspasari,Business Analyst,1.617634e+09,57739,Ambon,https://github.com/corneliahandayani,0898336695,CH-Komoro-UHP,corneliahandayani@uhp.com,2021-04-05 14:47:15
3,aeb6d817-25f3-4867-8a74-8d92e0a0f633,Soleh,Rajasa,1991-11-04,"Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156",+62 (418) 329-4756,Eritrea,Perum Tampubolon Yuliarti,DevOps Engineer,1.617634e+09,76156,Lubuklinggau,https://github.com/solehrajasa,04183294756,SR-Eritrea-PTY,solehrajasa@pty.com,2021-04-05 14:47:14
4,1fdabdd9-5444-4c97-87b2-fe8833ad0d27,Vivi,Astuti,2003-01-22,"Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...",0812511835,Aljazair,PT Hardiansyah Rahimah,Data Analyst,1.617634e+09,70074,Kediri,https://github.com/viviastuti,0812511835,VA-Aljazair-PHR,viviastuti@phr.com,2021-04-05 14:46:50
